In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\andy\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
from numpy.random import seed
seed(99)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
target = df['koi_disposition']
target.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(target)
encoded_y = label_encoder.transform(target)
encoded_y

array([1, 2, 2, ..., 0, 2, 2])

In [9]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, one_hot_y, random_state=99)

In [11]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth
5136,94.381630,171.336200,0.5930,11.87000,91.2
6214,24.238242,136.616569,0.4260,4.16665,461960.0
5863,2.705816,132.537260,0.6199,6.56000,46.9
2100,8.589696,174.962500,1.1970,8.47100,245.1
3977,2.401752,133.711919,5.2500,2.08350,25044.0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

# Train the Model



In [16]:
model = Sequential()
model.add(Dense(units=12, activation='relu', input_dim=5))
model.add(Dense(units=12, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(
    X_train_minmax,
    y_train_minmax,
    epochs=250,
    shuffle=True,
    verbose=2)

Train on 5243 samples
Epoch 1/250
5243/5243 - 0s - loss: 1.0556 - accuracy: 0.4986
Epoch 2/250
5243/5243 - 0s - loss: 1.0199 - accuracy: 0.5039
Epoch 3/250
5243/5243 - 0s - loss: 0.9915 - accuracy: 0.5041
Epoch 4/250
5243/5243 - 0s - loss: 0.9639 - accuracy: 0.5037
Epoch 5/250
5243/5243 - 0s - loss: 0.9382 - accuracy: 0.5087
Epoch 6/250
5243/5243 - 0s - loss: 0.9177 - accuracy: 0.5241
Epoch 7/250
5243/5243 - 0s - loss: 0.9031 - accuracy: 0.5417
Epoch 8/250
5243/5243 - 0s - loss: 0.8911 - accuracy: 0.5545
Epoch 9/250
5243/5243 - 0s - loss: 0.8808 - accuracy: 0.5646
Epoch 10/250
5243/5243 - 0s - loss: 0.8732 - accuracy: 0.5724
Epoch 11/250
5243/5243 - 0s - loss: 0.8652 - accuracy: 0.5768
Epoch 12/250
5243/5243 - 0s - loss: 0.8572 - accuracy: 0.5852
Epoch 13/250
5243/5243 - 0s - loss: 0.8512 - accuracy: 0.5905
Epoch 14/250
5243/5243 - 0s - loss: 0.8463 - accuracy: 0.5920
Epoch 15/250
5243/5243 - 0s - loss: 0.8427 - accuracy: 0.5949
Epoch 16/250
5243/5243 - 0s - loss: 0.8370 - accuracy: 0.

Epoch 133/250
5243/5243 - 0s - loss: 0.7295 - accuracy: 0.6666
Epoch 134/250
5243/5243 - 0s - loss: 0.7287 - accuracy: 0.6636
Epoch 135/250
5243/5243 - 0s - loss: 0.7302 - accuracy: 0.6622
Epoch 136/250
5243/5243 - 0s - loss: 0.7289 - accuracy: 0.6636
Epoch 137/250
5243/5243 - 0s - loss: 0.7281 - accuracy: 0.6655
Epoch 138/250
5243/5243 - 0s - loss: 0.7274 - accuracy: 0.6670
Epoch 139/250
5243/5243 - 0s - loss: 0.7295 - accuracy: 0.6664
Epoch 140/250
5243/5243 - 0s - loss: 0.7279 - accuracy: 0.6645
Epoch 141/250
5243/5243 - 0s - loss: 0.7279 - accuracy: 0.6662
Epoch 142/250
5243/5243 - 0s - loss: 0.7250 - accuracy: 0.6664
Epoch 143/250
5243/5243 - 0s - loss: 0.7273 - accuracy: 0.6653
Epoch 144/250
5243/5243 - 0s - loss: 0.7265 - accuracy: 0.6662
Epoch 145/250
5243/5243 - 0s - loss: 0.7255 - accuracy: 0.6706
Epoch 146/250
5243/5243 - 0s - loss: 0.7262 - accuracy: 0.6674
Epoch 147/250
5243/5243 - 0s - loss: 0.7237 - accuracy: 0.6666
Epoch 148/250
5243/5243 - 0s - loss: 0.7231 - accuracy:

In [20]:
model_loss, model_accuracy = model.evaluate(X_train_minmax, y_train_minmax, verbose=2)
print(f"Training Data Loss: {model_loss}, Training Data Accuracy: {model_accuracy}")

model_loss, model_accuracy = model.evaluate(X_test_minmax, y_test_minmax, verbose=2)
print(f"Test Data Loss: {model_loss}, Test Data Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.7109 - accuracy: 0.6685
Training Data Loss: 0.7108517847806016, Training Data Accuracy: 0.668510377407074
1748/1748 - 0s - loss: 0.7096 - accuracy: 0.6693
Test Data Loss: 0.7096232479988028, Test Data Accuracy: 0.6693363785743713


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [21]:
model

# Save the Model

In [23]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'andy_spatz.sav'
# neuralnet = model
# joblib.dump(neuralnet, filename)

model.save('neuralnetmodel.h5')